# Data encoding

In [1]:
from os import getcwd, chdir

if getcwd().endswith('notebooks'):
    chdir('..')

In [3]:
from data.getDataset import getCognatesSet, getIteration
from data.vocab import computeInferenceData_Samples, computeInferenceData_Cognates, wordsToOneHots
from random import randint

raw_cognates = getCognatesSet()
__random_start_index = randint(0, len(raw_cognates['french'])-6)
cognates = computeInferenceData_Cognates({lang: wordsToOneHots(raw_cognates[lang]) for lang in raw_cognates})['french']
raw_cognates = raw_cognates['french'][__random_start_index: __random_start_index + 5]
raw_samples = getIteration(1)[__random_start_index: __random_start_index + 5]
samples = computeInferenceData_Samples(wordsToOneHots(raw_samples)) #TODO: simplify the data loading

## Encoding

## Samples encoding: `InferenceData_Source` type

This type refers to a tuple of three elements:
- an IntTensor `S` of shape $\left(\max \{|x|, x \in \textrm{batch}\} + 2, c, b\right)$. For all $0 \leq i < c$ and $0 \leq j < b$, `S[:, i, j]` represents one sample among the $b$ ones which are linked with the $i$-th cognate pair. It is represented along the first axis by tokens encoded with one-hot indexes and the sequence is opened by the `SOS_TOKEN` and the `EOS_TOKEN`.
- a cpu ByteTensor `L` of shape $\left( c, b \right)$ containing the length of each samples with the boundaries token. It is defined such that `S[L[i, j]:, i, j]` is a list of the `PADDING_TOKEN`'s one-hot indices. Therefore, `L[i, j]` = $|x_{(i,j)}| + 2$, if we note $x_{(i,j)}$ as the raw sample (without the boundaries token) represented at the position (i, j) in `S`.
- `n`: the max of `L` (if the tuple is correctly defined, then `n = S.size()[0]`)

In [4]:
print(raw_samples)
print(samples[0][...,0].T)
print(samples[0].size())
print('\n' + "Samples' length (without boundaries):", str([len(c) for c in raw_samples]))
print("Samples' length (with boundaries):", samples[1][:,0])
print("Max sample length with boundaries:", samples[2])

['ɔpɨradʊ', 'ɔpɨɾaθjɔ', 'ɔpɨɾasjɔ', 'ɔpɨɾator', 'ɔpɛrkylʊ']
tensor([[58, 30, 13, 36, 14,  0,  2, 43, 57, 59],
        [58, 30, 13, 36, 40,  0, 56,  7, 30, 57],
        [58, 30, 13, 36, 40,  0, 15,  7, 30, 57],
        [58, 30, 13, 36, 40,  0, 16, 12, 14, 57],
        [58, 30, 13, 32, 14,  8, 21,  9, 43, 57]], dtype=torch.int32)
torch.Size([10, 5, 1])

Samples' length (without boundaries): [7, 8, 8, 8, 8]
Samples' length (with boundaries): tensor([ 9, 10, 10, 10, 10])
Max sample length with boundaries: 10


## Cognates encoding: `InferenceData_Targets` type

This type of data is defined by a tuple similar with `InferenceData_Source`, excepted that the `EOS_TOKEN` is here removed from the first IntTensor, which involves that the sequences lengths are reduced by one, compared to the sequences in the previous type. Therefore, we can sum up its three elements in the following list:
- `S`: an IntTensor of shape $\left( \max \{ |y_l|, y_l\in \textrm{batch}_l \} + 1, c \right)$
- `L`: a cpu ByteTensor of shape $(c)$ `L[i]` = $|y_{l, i}| + 1$
- `n`: the max of `L` (if the tuple is correctly defined, then `n = S.size()[0]` )

In [5]:
print(raw_cognates)
print(cognates[0].T)
print(cognates[0].size())
print('\n' + "Cognates' length (without SOS token):", str([len(c) for c in raw_cognates]))
print("Cognates' length (with SOS token):", cognates[1])
print("Max cognate length with SOS token:", cognates[2])

['opeʁˈe', 'opeʁasjˈɔ̃', 'opeʁasjˈɔ̃', 'opeʁatˈœʁ', 'opɛʁkˈyl']
tensor([[58,  3,  1,  ..., 59, 59, 59],
        [58,  0,  1,  ..., 59, 59, 59],
        [58,  0,  1,  ..., 59, 59, 59],
        ...,
        [58, 22, 51,  ..., 59, 59, 59],
        [58,  2, 51,  ..., 59, 59, 59],
        [58,  2, 51,  ..., 59, 59, 59]], dtype=torch.int32)
torch.Size([19, 3213])

Cognates' length (without SOS token): [6, 10, 10, 9, 8]
Cognates' length (with SOS token): tensor([6, 6, 8,  ..., 5, 4, 4])
Max cognate length with SOS token: 19
